In [9]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [20]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada.html', 'w') as fo:
    fo.write(article)
    

In [47]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('List_of_postal_codes_of_Canada.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('List_of_postal_codes_of_Canada.csv', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode, borough, neighbourhood = [td.text.strip() for td in tds[:3]]
        print(', '.join([postcode, borough, neighbourhood]), file=fo)

In [66]:
colnames=['Postcode', 'Borough', 'Neighbourhood']
df = pd.read_csv('List_of_postal_codes_of_Canada.csv',names=colnames)
df['Borough']=df['Borough'].str.strip()
df['Neighbourhood'] = df['Neighbourhood'].str.strip()
df.shape

(287, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [64]:
df1 = df[df['Borough'] != 'Not assigned']
df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [65]:
df1.shape

(210, 3)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [71]:
df2=df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.



In [73]:
df2.Neighbourhood.loc[df2.Neighbourhood=='Not assigned']=df2.Borough
len(df2[df2.Neighbourhood=='Not assigned'])

0

In [74]:
df2.shape

(103, 3)

In [87]:
from arcgis.geocoding import geocode


In [107]:
g = geocoder.arcgis('{}, Toronto, Ontario'.format('M1B')) 

In [108]:
(g.lat,g.lng)

(43.811525000000074, -79.19551746399998)

In [112]:
import time
start = time.time()
adict={}

for i in df2.Postcode:
    g = geocoder.arcgis('{},Toronto, Ontario, Canada'.format(i))    
    adict[i]=[g.lat, g.lng]
    print(g.lat, g.lng)
adict

43.811525000000074 -79.19551746399998
43.78566500000005 -79.15872457299997
43.76581500000003 -79.17519294699997
43.768369121000035 -79.21758999999997
43.76968799900004 -79.23943999999995
43.74312500000008 -79.23174973599998
43.72627568400003 -79.26362499999993
43.71305350000006 -79.28505499999994
43.724234575000025 -79.22792499999997
43.69677000000007 -79.25996735299998
43.759975000000054 -79.26897418299995
43.750710464000065 -79.30055999999996
43.79394000000008 -79.26797613999997
43.78472500000004 -79.29906603299997
43.81768500000004 -79.28018721399997
43.80088292800008 -79.32073999999994
43.83421500000003 -79.21670085099998
43.80284500000005 -79.35620744999994
43.780970000000025 -79.34781328099996
43.781015000000025 -79.38052867199997
43.75709511900004 -79.38031999999998
43.79147500000005 -79.41360487299994
43.76816500000007 -79.40741984599998
43.74789500000003 -79.39991893899997
43.77769500000005 -79.44579657299994
43.75242000000003 -79.32924245299995


Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Max retries exceeded with url: /arcgis/rest/services/World/GeocodeServer/find?f=json&text=M3B%2CToronto%2C+Ontario%2C+Canada&maxLocations=1 (Caused by ProxyError('Cannot connect to proxy.', timeout('_ssl.c:1059: The handshake operation timed out')))


None None
43.72142500000007 -79.34345422799998
43.75787500000007 -79.44868764799998
43.76476500000007 -79.48809440699995
43.73868500000003 -79.46731999999997


Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Max retries exceeded with url: /arcgis/rest/services/World/GeocodeServer/find?f=json&text=M3L%2CToronto%2C+Ontario%2C+Canada&maxLocations=1 (Caused by ProxyError('Cannot connect to proxy.', timeout('_ssl.c:1059: The handshake operation timed out')))


None None
43.73369000000008 -79.49674024199999
43.75537065200007 -79.51958999999994
43.73060024600005 -79.31326499999994
43.707535000000064 -79.31177329699995
43.689640000000054 -79.30687387799998
43.67653121600006 -79.29542499999997
43.70949500000006 -79.36398897099997
43.70124000000004 -79.34982482599997
43.688765344000046 -79.33417499999996
43.68317787500007 -79.35510499999998
43.66796500000004 -79.31466717299998
43.660628993000046 -79.33485499999995
43.72842000000003 -79.38713269199997
43.71275500000007 -79.38851449699996


...

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Max retries exceeded with url: /arcgis/rest/services/World/GeocodeServer/find?f=json&text=M9N%2CToronto%2C+Ontario%2C+Canada&maxLocations=1 (Caused by ProxyError('Cannot connect to proxy.', timeout('_ssl.c:1059: The handshake operation timed out')))


None None
43.69650500000006 -79.53025233799997
43.68681000000004 -79.55728354099995


Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Max retries exceeded with url: /arcgis/rest/services/World/GeocodeServer/find?f=json&text=M9V%2CToronto%2C+Ontario%2C+Canada&maxLocations=1 (Caused by ProxyError('Cannot connect to proxy.', timeout('_ssl.c:1059: The handshake operation timed out')))


None None
43.71174000000008 -79.57918134599998


{'M1B': [43.811525000000074, -79.19551746399998],
 'M1C': [43.78566500000005, -79.15872457299997],
 'M1E': [43.76581500000003, -79.17519294699997],
 'M1G': [43.768369121000035, -79.21758999999997],
 'M1H': [43.76968799900004, -79.23943999999995],
 'M1J': [43.74312500000008, -79.23174973599998],
 'M1K': [43.72627568400003, -79.26362499999993],
 'M1L': [43.71305350000006, -79.28505499999994],
 'M1M': [43.724234575000025, -79.22792499999997],
 'M1N': [43.69677000000007, -79.25996735299998],
 'M1P': [43.759975000000054, -79.26897418299995],
 'M1R': [43.750710464000065, -79.30055999999996],
 'M1S': [43.79394000000008, -79.26797613999997],
 'M1T': [43.78472500000004, -79.29906603299997],
 'M1V': [43.81768500000004, -79.28018721399997],
 'M1W': [43.80088292800008, -79.32073999999994],
 'M1X': [43.83421500000003, -79.21670085099998],
 'M2H': [43.80284500000005, -79.35620744999994],
 'M2J': [43.780970000000025, -79.34781328099996],
 'M2K': [43.781015000000025, -79.38052867199997],
 'M2L': [43.7

In [136]:
latlng=pd.DataFrame.from_dict(adict,orient='index').reset_index()
latlng.columns=['Postcode','Latitude','Longitude']

In [137]:
latlng.head()

,Postcode,Latitude,Longitude
0,M1B,43.811525,-79.195517
1,M1C,43.785665,-79.158725
2,M1E,43.765815,-79.175193
3,M1G,43.768369,-79.217590
4,M1H,43.769688,-79.239440


In [138]:
df3=pd.merge(df2,latlng,how='left', on=['Postcode'])
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [139]:
df3.to_csv('toronto_geo.csv')

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

In [140]:
df3.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

#### Borough Contains Toronto

In [141]:
torontoData = df3[df3['Borough'].str.contains("Toronto")]

In [142]:
torontoData.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676531,-79.295425
41,M4K,East Toronto,"The Danforth West,Riverdale",43.683178,-79.355105
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314667
43,M4M,East Toronto,Studio District,43.660629,-79.334855
44,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133


In [143]:
torontoData.shape

(39, 5)

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

* to add enough Markdown cells to explain what you decided to do and to report any observations you make.
* to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [144]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(torontoData['Borough'].unique()),
        torontoData.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


In [154]:
location = geocoder.arcgis('Toronto, Ontario, Canada') 
address_tr = 'Toronto, Ontario, Canada'
location.json
latitude_tr = location.lat
longitude_tr = location.lng
print('The geograpical coordinate of Toronto, Ontario, Canada are {}, {}.'.format(latitude_tr, longitude_tr))

The geograpical coordinate of Toronto, Ontario, Canada are 43.648690000000045, -79.38543999999996.


In [161]:
# create map of Toronto using latitude and longitude values
torontoData = torontoData.dropna()

In [162]:
map_tr = folium.Map(location=[latitude_tr, longitude_tr], zoom_start=12)

# add markers to map
for lat, lng, label in zip(torontoData['Latitude'], torontoData['Longitude'], torontoData['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5).add_to(map_tr)  
    
map_tr

New York City Boroughs from kaggle

In [233]:
nyc_gallery=pd.read_csv("ART_GALLERY.csv")
nyc_gallery.the_geom = nyc_gallery.the_geom.str.replace("POINT","")
nyc_gallery.the_geom = nyc_gallery.the_geom.str.replace("(","")
nyc_gallery.the_geom = nyc_gallery.the_geom.str.replace(")","")
nyc_gallery['lat'] = nyc_gallery['the_geom'].apply( lambda x: x.split(' ')[2] )
nyc_gallery['lng'] = nyc_gallery['the_geom'].apply( lambda x: x.split(' ')[1] )
nyc_gallery = nyc_gallery.drop(['the_geom'],axis=1)

In [240]:
nyc_gallery[nyc_gallery['ADDRESS2'].notna()]

,NAME,TEL,URL,ADDRESS1,ADDRESS2,CITY,ZIP,lat,lng
7,Pace MacGill Gallery,(212) 759-7999,http://www.pacemacgill.com/exhibitions.html,32 E. 57th St.,9th Fl.,New York,10022,40.76194965113893,-73.97202174911459
8,Pace Prints & Primitive Gallery,(212) 421-3237,http://www.paceprints.com/,32 E. 57th St.,3rd Fl.,New York,10022,40.76183646853851,-73.97211461855828
10,Pace Wildenstein,(212) 421-3292,http://www.pacewildenstein.com/uploads/exhibit...,32 E 57th St,Fl 4,New York,10022,40.761886074501135,-73.97207691758459
26,Peder Bonnier Inc,(646) 638-0878,http://web2.userinstinct.com/30816857-peder-bo...,206 W 23rd St,Fl 4,New York,10011,40.7440599458188,-73.99623914232185
30,Peter Rose Gallery,(212) 759-8173,http://peterrosegallery.com/,200 E 58th St,Apt 20C,New York,10022,40.760575546804205,-73.96690494675263
...,...,...,...,...,...,...,...,...,...
872,Miller Laurence Gallery Inc,(212) 397-3930,http://www.laurencemillergallery.com/,20 W 57th St,Lbby,New York,10019,40.76323164644723,-73.97553261570835
875,Mip Art Realty Inc,(212) 431-9474,http://www.city-data.com/businesses/578718041-...,83 Mercer St,Fl 5,New York,10012,40.72296987769243,-74.00028349944168
886,Morales David,(212) 564-2335,http://www.nyc.com/arts__attractions/morales_d...,325 W 38th St,Lbby,New York,10018,40.755395881808944,-73.99280897511609
887,Morgan Art Consulting,(212) 447-0490,http://morganartconsulting.com/,303 Park Avenue South,#1204,New York,10010,40.74025449638514,-73.98578003114238


It turns out that Address2 is mainly floor number, so not very useful. I decide to drop it. 
URL is also not useful in this case, so drop

In [243]:
nyc_gallery.drop(['ADDRESS2', 'URL'], axis=1)

,NAME,TEL,ADDRESS1,CITY,ZIP,lat,lng
0,O'reilly William & Co Ltd,(212) 396-1822,52 E 76th St,New York,10021,40.773800871637576,-73.96273074561996
1,Organization of Independent Artists - Gallery 402,(212) 219-9213,19 Hudson St.,New York,10013,40.71646850931738,-74.0093857886298
2,Owen Gallery,(212) 879-2415,19 E 75th St,New York,10021,40.7740008522409,-73.96435178145856
3,P P O W Gallerie,(212) 647-1044,511 W 25th St,New York,10001,40.74958540084406,-74.00389261044238
4,P P O W Inc,(212) 941-8642,476 Broome St,New York,10013,40.72290742771331,-74.00176389614576
...,...,...,...,...,...,...,...
912,Odyssia Gallery,(212) 486-7338,305 E 61st St,New York,10021,40.7618747512849,-73.96269813635554
913,Old Print Shop Inc,(212) 683-3950,150 Lexington Ave,New York,10016,40.74367322071215,-73.98194844064098
914,Omniguard Art Guarantee,(212) 577-9000,115 Broadway,New York,10006,40.708938811665874,-74.0114722358904
915,One Twenty Eight,(212) 674-0244,128 Rivington St,New York,10002,40.719806519486134,-73.9869130305955


In [244]:
df3.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64